## Sales Analysis

### Aim

This Notebook aims for analysing the following items:

**SalesPreviousMonth**:sales analysis from previous month [Product / Sales]

**SalesPreviousYear**:sales analysis from same month last year [Product / Sales]

**SalesRolling12Months**:sales analysis of trend over the last 12 months [Product / Sales]

**SalesYearToDate**:sales analysis of trend year to date [Product / Sales]

**SalesSummary**:summary of all the above

**SalesProductVsCategory**:sales analysis of this specific product vs trend for category [Product / Sales]

**SalesTerritory**:sales analysis of territory [Product / Sales]

**SalesCustomerType**:sales analysis of direct vs reseller [Product / Sales / Companies / Contacts]

**SalesProductReviews**:analysing all the reviews for that month [Fact_AiSocialReviews]

**SalesInsights**:insights from the different sales analysis types above

**SalesActions**:actions from analysing all of the above information


### Installing package

In [1]:
%pip install openai

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.3/327.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Not uninstalling typing-extensions at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-f771207c-060f-43fd-83e6-3cc53ee213f2
    Can't uninstall 'typing_extensions'. No f

### Import Library

In [19]:
from pyspark.sql.functions import col, desc, to_date, add_months, lit, trim, concat, sum
from pyspark.sql import Row
from openai import AzureOpenAI, RateLimitError, OpenAIError, APIError
import sys  
import io
import time
from itertools import count

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 26, Finished, Available, Finished)

### Setting the test example

In [20]:
TEST = True # set to True if you want to run the test
product_model = "Cycling Cap" # test example product model
end_date_str = "2014-05" # test example end period

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 27, Finished, Available, Finished)

## Sales Analysis by Product Model

In [21]:
df_sales = spark.sql("SELECT SalesOrderID,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,OrderDate,date_format(OrderDate, 'yyyy') AS OrderYear,date_format(OrderDate, 'MM') AS OrderMonth,concat(date_format(OrderDate, 'yyyy'),'-',date_format(OrderDate, 'MM')) AS YearMonth FROM AdventureWorks_Lakehouse.Ops_Gold.fact_sales")
df_products=spark.sql("SELECT * FROM AdventureWorks_Lakehouse.Ops_Gold.dim_products")
df_salesanalysis=df_sales.join(df_products,'ProductID',"left")
display(df_salesanalysis)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b728fdd4-d580-4092-8899-5339fa003109)

In [22]:
from pyspark.sql.functions import count,sum
df_salesanalysis_final=df_salesanalysis.groupBy('ProductModel','YearMonth').agg(sum('OrderQty').alias('TotalUnitsSold'),sum('LineTotal').alias('Revenue(USD)'))
df_salesanalysis_final = df_salesanalysis_final.withColumn('YearMonth', trim(col('YearMonth')).cast('string'))  
df_salesanalysis_final = df_salesanalysis_final.withColumn(  
    'YearMonthDate',  
    to_date(concat(col('YearMonth'), lit('-01')), 'yyyy-MM-dd')  
)  
df_salesanalysis_final_sorted = df_salesanalysis_final.orderBy('ProductModel', desc('YearMonth')).dropna()
display(df_salesanalysis_final_sorted)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 29, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8a44588f-5dcc-4d84-93fe-4f6762b5a527)

### Function to refine the column and Loop the YearMonth back for 1 year.

In [23]:
def refine_selection(df, col_name, selection,end_date_str):
    end_date = to_date(lit(end_date_str), 'yyyy-MM')  
    start_date = add_months(end_date, -12)  
    df_filtered = df.filter(  
        (col(col_name) == selection) &  
        (col('YearMonthDate') >= start_date) &  
        (col('YearMonthDate') <= end_date)  
    ).orderBy('YearMonthDate')
    return df_filtered

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 30, Finished, Available, Finished)

#### Test

In [24]:
if TEST:
    df_filtered = refine_selection(df = df_salesanalysis_final_sorted, col_name = "ProductModel", selection = product_model,end_date_str=end_date_str)
    df_filtered.show()

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 31, Finished, Available, Finished)

+------------+---------+--------------+------------+-------------+
|ProductModel|YearMonth|TotalUnitsSold|Revenue(USD)|YearMonthDate|
+------------+---------+--------------+------------+-------------+
| Cycling Cap|  2013-05|           261| 1295.769500|   2013-05-01|
| Cycling Cap|  2013-06|           465| 2611.340100|   2013-06-01|
| Cycling Cap|  2013-07|           526| 3331.747000|   2013-07-01|
| Cycling Cap|  2013-08|           354| 2450.508000|   2013-08-01|
| Cycling Cap|  2013-09|           440| 2879.483000|   2013-09-01|
| Cycling Cap|  2013-10|           406| 2738.841600|   2013-10-01|
| Cycling Cap|  2013-11|           319| 2429.284800|   2013-11-01|
| Cycling Cap|  2013-12|           301| 2281.662000|   2013-12-01|
| Cycling Cap|  2014-01|           365| 2727.677700|   2014-01-01|
| Cycling Cap|  2014-02|           180| 1618.200000|   2014-02-01|
| Cycling Cap|  2014-03|           663| 4219.746800|   2014-03-01|
| Cycling Cap|  2014-04|           193| 1735.070000|   2014-04

### Function to capture the output of df.show() as a string

In [25]:
def get_dataframe_show_string(df, truncate=True):  
    buffer = io.StringIO()  
    num_rows = df.count()
    old_stdout = sys.stdout  
    try:  
        sys.stdout = buffer  
        df.show(num_rows, truncate=truncate)  
        output = buffer.getvalue()  
    finally:  
        sys.stdout = old_stdout  
    return output  

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 32, Finished, Available, Finished)

#### Test 

In [26]:
if TEST:
    df_output_string = get_dataframe_show_string(df_filtered, truncate=False) 
    print (df_output_string)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 33, Finished, Available, Finished)

+------------+---------+--------------+------------+-------------+
|ProductModel|YearMonth|TotalUnitsSold|Revenue(USD)|YearMonthDate|
+------------+---------+--------------+------------+-------------+
|Cycling Cap |2013-05  |261           |1295.769500 |2013-05-01   |
|Cycling Cap |2013-06  |465           |2611.340100 |2013-06-01   |
|Cycling Cap |2013-07  |526           |3331.747000 |2013-07-01   |
|Cycling Cap |2013-08  |354           |2450.508000 |2013-08-01   |
|Cycling Cap |2013-09  |440           |2879.483000 |2013-09-01   |
|Cycling Cap |2013-10  |406           |2738.841600 |2013-10-01   |
|Cycling Cap |2013-11  |319           |2429.284800 |2013-11-01   |
|Cycling Cap |2013-12  |301           |2281.662000 |2013-12-01   |
|Cycling Cap |2014-01  |365           |2727.677700 |2014-01-01   |
|Cycling Cap |2014-02  |180           |1618.200000 |2014-02-01   |
|Cycling Cap |2014-03  |663           |4219.746800 |2014-03-01   |
|Cycling Cap |2014-04  |193           |1735.070000 |2014-04-01

### Function to call OpenAI service

In [27]:
# Remove before submission
ENDPOINT = "https://mango-bush-0a9e12903.5.azurestaticapps.net/api/v1"
API_KEY = "40ead0b8-b87b-4c13-88a4-efe54c700483"

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 34, Finished, Available, Finished)

In [28]:
# General GPT call function
def open_ai_call(system_message, user_message, max_retries=5, initial_wait_time=20):
    API_VERSION = "2024-02-01"
    MODEL_NAME = "gpt-4-turbo-2024-04-09"
    # MODEL_NAME = "gpt-4o"
    retry_count = 0
    wait_time = initial_wait_time

    client = AzureOpenAI(
        azure_endpoint=ENDPOINT,
        api_key=API_KEY,
        api_version=API_VERSION,
    )

    MESSAGES = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},]

    while retry_count < max_retries:
        try:
            completion = client.chat.completions.create(
                model=MODEL_NAME,
                messages=MESSAGES,
            )
            response = completion.choices[0].message.content
            print (response)
            return response

        except RateLimitError as e:
            print(f"Rate limit error (429) encountered: {e}. Retrying {retry_count + 1}/{max_retries} after {wait_time} seconds...")
            retry_count += 1
            time.sleep(wait_time)
            wait_time *= 2  

        except APIError as e:
            print(f"Server error (500) encountered: {e}. Retrying {retry_count + 1}/{max_retries} after {wait_time} seconds...")
            retry_count += 1
            time.sleep(wait_time)
            wait_time *= 2  

        except OpenAIError as e:
            print("An unexpected OpenAI error occurred:", e)
            break  

    return "N.A."

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 35, Finished, Available, Finished)

### Function to concatenate the strings

In [29]:
# Concatenate information strings
def concat_info_with_annotation(info_list):    
    output_string = ""
    for i, item in enumerate(info_list, start=1):
        output_string += f"information {i}:\n{item}\n"
    print ("output string from info list:\n", output_string)
    return (output_string)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 36, Finished, Available, Finished)

### Function to analyse product model sales

In [30]:
# SalesPreviousMonth
def SalesPreviousMonth(query_table):
    system_message_ = "Using this sales data below 'Compare the sales of current month from last month' The last line in the table is the first month. Just provide a summary with the key data. *ONE SENTENCE ONLY*. *NO DETAILED ANALYSIS*."
    sales_analysis_previous_month = open_ai_call(system_message = system_message_, user_message=query_table)
    return sales_analysis_previous_month
# SalesPreviousYear
def SalesPreviousYear(query_table):
    system_message = "compare the sales from the current month to the same month in the previous year, the last line in the table is the first month. Just provide a summary with the key data. *ONE SENTENCE ONLY*. *NO DETAILED ANALYSIS*."
    sales_analysis_previous_year = open_ai_call(system_message = system_message, user_message=query_table)
    return sales_analysis_previous_year
# SalesRolling12Months
def SalesRolling12Months(query_table):
    system_message = "analyse the sales rolling trend over the last 12 month. Please give high level analysis. Just provide a summary with the key data. *ONE SENTENCE ONLY*. *NO DETAILED ANALYSIS*."
    sales_rolling_12_month = open_ai_call(system_message = system_message, user_message=query_table)
    return sales_rolling_12_month
# SalesYearToDate
def SalesYearToDate(query_table):
    system_message = "Provide sales analysis of trend for the current year starting from Jan of the same year to date where the current month is at the last line of the table.\
    Comment in the reality and projection. Please give high level analysis. Just provide a summary with the key data. *ONE SENTENCE ONLY*. *NO DETAILED ANALYSIS*."
    sales_year_to_date = open_ai_call(system_message = system_message, user_message=query_table)
    return sales_year_to_date
# SalesSummary
def SalesSummary(info_for_SalesSummary):
    system_message = "You are writing a short paragraph for a sales and marketing director of the analysis.\
                    You have done on your sales data for one of your products.\
                    You will be given four summary points.\
                    You need to extract the relevant information out of what was provided and provide a very short summary of what has happened this month and the general trend.\
                    Please keep the paragraph short."
    sales_summary = open_ai_call(system_message = system_message, user_message=info_for_SalesSummary)
    return sales_summary

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 37, Finished, Available, Finished)

#### Test

In [31]:
if TEST:
    product_model = "Cycling Cap"
    end_date_str = "2014-05"
    df_filtered = refine_selection(df = df_salesanalysis_final_sorted, col_name = "ProductModel", selection = product_model,end_date_str=end_date_str)
    query_table_string = get_dataframe_show_string(df_filtered, truncate=False) 
    print ("sales_analysis_previous_month")
    sales_analysis_previous_month = SalesPreviousMonth(query_table_string)
    print ("\n")
    print ("sales_analysis_previous_year")
    sales_analysis_previous_year = SalesPreviousYear(query_table_string)
    print ("\n")
    print ("sales_rolling_12_month")
    sales_rolling_12_month = SalesRolling12Months(query_table_string)
    print ("\n")
    print ("sales_year_to_date")
    sales_year_to_date = SalesYearToDate(query_table_string)
    print ("\n")
    print ("sales_summary")
    info_list = [sales_analysis_previous_month,sales_analysis_previous_year,sales_rolling_12_month,sales_year_to_date]
    info_for_SalesSummary = concat_info_with_annotation(info_list)
    sales_summary = SalesSummary(info_for_SalesSummary)
    print ("\n")


StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 38, Finished, Available, Finished)

sales_analysis_previous_month
In May 2014, the Cycling Cap sold 456 units generating $3294.20 in revenue, compared to 193 units sold and $1735.07 in revenue in April 2014.


sales_analysis_previous_year
In May 2014, sales increased to 456 units and revenue reached $3294.20 compared to May 2013 with 261 units and $1295.77 in revenue.


sales_rolling_12_month
Over the past 12 months, sales of Cycling Caps fluctuated reaching a peak in July 2013 and March 2014, with a notable drop in February 2014, while revenue generally followed the same trend as unit sales.


sales_year_to_date
The sales have been generally fluctuating, peaking in March 2014 with 663 units sold and revenue of $4219.75, demonstrating significant variability across the months with a significant dip in February 2014.


sales_summary
output string from info list:
 information 1:
In May 2014, the Cycling Cap sold 456 units generating $3294.20 in revenue, compared to 193 units sold and $1735.07 in revenue in April 2014.
info

## Sales Analysis by Product Model & Categories

In [32]:
from pyspark.sql.functions import count,sum
df_salesanalysis_productcategory=df_salesanalysis.groupBy('ProductModel','ProductCategory','ProductSubCategory','YearMonth').agg(sum('OrderQty').alias('TotalUnitsSold'),sum('LineTotal').alias('Revenue(USD)'))
df_salesanalysis_productcategory = df_salesanalysis_productcategory.withColumn('YearMonth', trim(col('YearMonth')).cast('string'))  
df_salesanalysis_productcategory = df_salesanalysis_productcategory.withColumn(  
    'YearMonthDate',  
    to_date(concat(col('YearMonth'), lit('-01')), 'yyyy-MM-dd')  
)  
display(df_salesanalysis_productcategory)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 39, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 27839f4a-8610-44b0-b6e3-6b0d06faf1ae)

### Mapping the product model with its corresponding category and product id

In [33]:
map_model_to_category = {row['ProductModel']:row['ProductCategory'] for row in df_salesanalysis_productcategory.collect()}
print(map_model_to_category)
map_model_to_id = {row['ProductModel']:row['ProductID'] for row in df_salesanalysis.collect()}
print(map_model_to_id)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 40, Finished, Available, Finished)

{'Road-350-W': 'Bikes', 'Half-Finger Gloves': 'Clothing', 'HL Mountain Frame': 'Components', 'HL Road Tire': 'Accessories', 'Road-750': 'Bikes', 'Road-550-W': 'Bikes', None: None, 'ML Road Tire': 'Accessories', 'ML Mountain Tire': 'Accessories', 'ML Road Frame-W': 'Components', 'Sport-100': 'Accessories', 'Touring-2000': 'Bikes', 'HL Touring Seat/Saddle': 'Components', 'HL Touring Frame': 'Components', 'Classic Vest': 'Clothing', "Women's Mountain Shorts": 'Clothing', 'Long-Sleeve Logo Jersey': 'Clothing', 'ML Mountain Handlebars': 'Components', 'Touring-1000': 'Bikes', 'Front Brakes': 'Components', 'Chain': 'Components', 'ML Road Pedal': 'Components', 'ML Mountain Seat/Saddle 2': 'Components', 'Road Bottle Cage': 'Accessories', 'Cycling Cap': 'Clothing', 'HL Road Frame': 'Components', 'Mountain Bottle Cage': 'Accessories', 'Touring Tire': 'Accessories', 'Hitch Rack - 4-Bike': 'Accessories', 'LL Mountain Seat/Saddle 2': 'Components', 'HL Mountain Seat/Saddle 2': 'Components', 'HL Mount

### Function to concatenate both product model and the corresponding category table as one string

In [34]:
def category_table_AsString(product_model, end_date_str):
    global map_model_to_category
    category_name = map_model_to_category[product_model]
    df_category_filtered = refine_selection(df = df_salesanalysis_productcategory, col_name = "ProductCategory", selection = category_name, end_date_str = end_date_str)
    df_category_filtered = df_category_filtered.groupBy('YearMonth').agg(  
        sum('TotalUnitsSold').alias('TotalUnitsSoldSum'), 
        sum('Revenue(USD)').alias('TotalRevenue(USD)')
        ).orderBy('YearMonth') 
    df_output_string = get_dataframe_show_string(df_category_filtered, truncate=True) 
    return df_output_string

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 41, Finished, Available, Finished)

#### Test

In [35]:
if TEST:
    category_table_string = category_table_AsString(product_model, end_date_str)
    print (category_table_AsString(product_model, end_date_str))

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 42, Finished, Available, Finished)

+---------+-----------------+-----------------+
|YearMonth|TotalUnitsSoldSum|TotalRevenue(USD)|
+---------+-----------------+-----------------+
|  2013-05|             3205|     82242.166500|
|  2013-06|             5114|    135511.215900|
|  2013-07|             5139|    143796.242500|
|  2013-08|             3291|     95035.533900|
|  2013-09|             4052|    115857.967300|
|  2013-10|             3844|    111772.641800|
|  2013-11|             2370|     73999.101600|
|  2013-12|             2629|     82177.869500|
|  2014-01|             2708|     84562.744700|
|  2014-02|              695|     26122.550000|
|  2014-03|             6379|    183243.344500|
|  2014-04|              822|     30303.330000|
|  2014-05|             3967|    119667.839800|
+---------+-----------------+-----------------+




### Function for comparing model and category

In [36]:
def SalesProductVsCategory(query_table, category_query_table):
    system_message = "Please provide analysis summary comparing the Product Model\
                        and its corresponding product category over the same period Starting from Jan of the year.\
                        Please compare the trend, peaks and \
                        Just provide a summary with the key data.\
                        *ONE SENTENCE ONLY*. *NO DETAILED ANALYSIS*."
    user_message = f"This is the product model table for the year:\n{query_table}\n\
                     This is its corresponding category table for the year:\n{category_query_table}"
    sales_product_vs_category = open_ai_call(system_message, user_message, max_retries=5, initial_wait_time=20)
    return sales_product_vs_category

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 43, Finished, Available, Finished)

#### Test

In [37]:
if TEST:
    df_filtered = refine_selection(df = df_salesanalysis_final_sorted, col_name = "ProductModel", selection = product_model,end_date_str=end_date_str)
    query_table_string = get_dataframe_show_string(df_filtered, truncate=False) 
    category_table_string = category_table_AsString(product_model, end_date_str)
    sales_product_vs_category = SalesProductVsCategory(query_table = query_table_string, category_query_table = category_table_string)


StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 44, Finished, Available, Finished)

The Cycling Cap shows inconsistent sales trends with peaks in July 2013 and March 2014, contributing minorly to the overall category sales, which generally have broader variations and reach a peak in June and July 2013 then March 2014.


## Product Sales by Territory

### Table for Territorial Sales

In [38]:
df_sales = spark.sql("SELECT SalesOrderID,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,OrderDate,date_format(OrderDate, 'yyyy') AS OrderYear,date_format(OrderDate, 'MM') AS OrderMonth,concat(date_format(OrderDate, 'yyyy'),'-',date_format(OrderDate, 'MM')) AS YearMonth,SalesTerritoryID,SalesTerritoryName,SalesTerritoryCountry,SalesTerritoryGroup FROM AdventureWorks_Lakehouse.Ops_Gold.fact_sales")
df_sales_analysis_territory = df_sales.groupBy('SalesTerritoryName','SalesTerritoryCountry','SalesTerritoryGroup','YearMonth').agg(sum('OrderQty').alias('TotalUnitsSold'),sum('LineTotal').alias('Revenue(USD)'))
display(df_sales_analysis_territory)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 45, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ba7883e2-c32e-41b0-acd8-13bafda31fbc)

### Table for Territorial Sales by Product Model

In [39]:
from pyspark.sql.functions import count,sum
df_sales = spark.sql("SELECT SalesOrderID,ProductID,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,OrderDate,date_format(OrderDate, 'yyyy') AS OrderYear,date_format(OrderDate, 'MM') AS OrderMonth,concat(date_format(OrderDate, 'yyyy'),'-',date_format(OrderDate, 'MM')) AS YearMonth,SalesTerritoryID,SalesTerritoryName,SalesTerritoryCountry,SalesTerritoryGroup FROM AdventureWorks_Lakehouse.Ops_Gold.fact_sales")
df_products=spark.sql("SELECT * FROM AdventureWorks_Lakehouse.Ops_Gold.dim_products")
df_sales_analysis_territory_product=df_sales.join(df_products,'ProductID',"left")
df_sales_analysis_territory_productmodel = df_sales_analysis_territory_product.groupBy('ProductModel','SalesTerritoryName','SalesTerritoryCountry','SalesTerritoryGroup','YearMonth').agg(sum('OrderQty').alias('TotalUnitsSold'),sum('LineTotal').alias('Revenue(USD)'))
df_sales_analysis_territory_productmodel = df_sales_analysis_territory_productmodel.withColumn('YearMonth', trim(col('YearMonth')).cast('string'))  
df_sales_analysis_territory_productmodel = df_sales_analysis_territory_productmodel.withColumn('YearMonthDate',\
                                                        to_date(concat(col('YearMonth'), lit('-01')), 'yyyy-MM-dd')\
                                                                                                )  
display(df_sales_analysis_territory_productmodel)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 46, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4f19cdf2-7304-4c9f-9f47-87c80cbdaa6e)

#### Test

In [40]:
if TEST:
    df_territory_refined = refine_selection(df = df_sales_analysis_territory_productmodel, col_name="ProductModel", 
                                            selection=product_model, end_date_str = end_date_str)
    df_territory_refined.show()

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 47, Finished, Available, Finished)

+------------+------------------+---------------------+-------------------+---------+--------------+------------+-------------+
|ProductModel|SalesTerritoryName|SalesTerritoryCountry|SalesTerritoryGroup|YearMonth|TotalUnitsSold|Revenue(USD)|YearMonthDate|
+------------+------------------+---------------------+-------------------+---------+--------------+------------+-------------+
| Cycling Cap|         Southeast|                   US|      North America|  2013-05|            20|  107.880000|   2013-05-01|
| Cycling Cap|            Canada|                   CA|      North America|  2013-05|            27|  131.704000|   2013-05-01|
| Cycling Cap|            France|                   FR|             Europe|  2013-05|            19|   90.642100|   2013-05-01|
| Cycling Cap|         Northeast|                   US|      North America|  2013-05|            23|  120.084600|   2013-05-01|
| Cycling Cap|           Germany|                   DE|             Europe|  2013-05|            33|  16

### Function to analyse sales of territory

In [41]:
def SalesTerritory(territory_query_table):
    system_message = "Analyse the sales at different Territories of this specific ProductModel.\
                        Just provide a summary with the key data. \
                        *ONE SENTENCE ONLY*. *NO DETAILED ANALYSIS*."
    sales_territory = open_ai_call(system_message = system_message, user_message=territory_query_table)
    return sales_territory

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 48, Finished, Available, Finished)

#### Test

In [42]:
if TEST:
    df_territory_refined = refine_selection(df = df_sales_analysis_territory_productmodel, col_name="ProductModel", 
                                            selection=product_model, end_date_str = end_date_str)
    territory_query_table_string = get_dataframe_show_string(df_territory_refined,truncate=False)
    sales_territory = SalesTerritory(territory_query_table_string)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 49, Finished, Available, Finished)

The Cycling Cap sold across various territories from May 2013 to May 2014 with notable sales in the North America and Europe regions; Canada and Southwest US generally had the highest unit sales and revenue, while sales in the Pacific (Australia) also increased notably over time.


## Sales Direct vs Reseller

### Table for Direct and Reseller

In [43]:
df_sales = spark.sql("SELECT OrderQty,ProductID,PersonID,LineTotal,date_format(OrderDate, 'yyyy') AS OrderYear,date_format(OrderDate, 'MM') AS OrderMonth,concat(date_format(OrderDate, 'yyyy'),'-',date_format(OrderDate, 'MM')) AS YearMonth FROM AdventureWorks_Lakehouse.Ops_Gold.fact_sales")

df_sales_product= df_products=spark.sql("SELECT * FROM AdventureWorks_Lakehouse.Ops_Gold.dim_products")
df_sales_product=df_sales.join(df_products,'ProductID',"left")

df_customers=spark.sql("SELECT * FROM AdventureWorks_Lakehouse.Ops_Gold.dim_customers")
df_sales_product_customers=df_sales_product.join(df_customers,df_sales_product.PersonID==df_customers.PersonId,"left")
df_sales_product_customers_sellertype=df_sales_product_customers.groupby('ProductModel','CustomerType','YearMonth').agg(sum('OrderQty').alias('TotalUnitsSold'),sum('LineTotal').alias('TotalRevenue(USD)'))

df_sales_product_customers_sellertype = df_sales_product_customers_sellertype.withColumn('YearMonth', trim(col('YearMonth')).cast('string'))  
df_sales_product_customers_sellertype = df_sales_product_customers_sellertype.withColumn('YearMonthDate',\
                                                        to_date(concat(col('YearMonth'), lit('-01')), 'yyyy-MM-dd')\
                                                                                                )  

display(df_sales_product_customers_sellertype)


StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 50, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f2e6a116-89ae-406c-97c7-6be3e9610c02)

#### Test

In [44]:
if TEST:
    df_sellertype_refined = refine_selection(df = df_sales_product_customers_sellertype, col_name="ProductModel", 
                                            selection=product_model, end_date_str = end_date_str)
    df_sellertype_refined.show()

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 51, Finished, Available, Finished)

+------------+------------+---------+--------------+-----------------+-------------+
|ProductModel|CustomerType|YearMonth|TotalUnitsSold|TotalRevenue(USD)|YearMonthDate|
+------------+------------+---------+--------------+-----------------+-------------+
| Cycling Cap|    Reseller|  2013-05|           254|      1250.819500|   2013-05-01|
| Cycling Cap|        NULL|  2013-05|             5|        26.970000|   2013-05-01|
| Cycling Cap|      Direct|  2013-05|             2|        17.980000|   2013-05-01|
| Cycling Cap|    Reseller|  2013-06|           390|      2019.798100|   2013-06-01|
| Cycling Cap|        NULL|  2013-06|            23|       124.062000|   2013-06-01|
| Cycling Cap|      Direct|  2013-06|            52|       467.480000|   2013-06-01|
| Cycling Cap|      Direct|  2013-07|           165|      1483.350000|   2013-07-01|
| Cycling Cap|        NULL|  2013-07|            50|       242.528700|   2013-07-01|
| Cycling Cap|    Reseller|  2013-07|           311|      1605.86

### Function for analysing different seller types

In [45]:
def SalesCustomerType(customer_query_table):
    system_message = "Analyse the sales throughout the year of this specific ProductModel. \
                Please analyse the trend throughout the year from the direct and resellers.\
                Please highlight the key information regarding the peaks and dips and trend for resell or direct sell.\
                *Do NOT need to Mention the Product ID\
                *ONE SENTENCE ONLY*. *NO DETAILED ANALYSIS*."
    sales_customer_type = open_ai_call(system_message = system_message, user_message=customer_query_table)
    return sales_customer_type

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 52, Finished, Available, Finished)

#### Test

In [46]:
if TEST:
    df_sellertype_refined = refine_selection(df = df_sales_product_customers_sellertype, col_name="ProductModel", 
                                            selection=product_model, end_date_str = end_date_str)
    sellertype_query_table_string = get_dataframe_show_string(df_sellertype_refined,truncate=False)
    sales_customer_type = SalesCustomerType(sellertype_query_table_string)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 53, Finished, Available, Finished)

Throughout the year, sales of the Cycling Cap to resellers generally outperform direct sales, with noticeable peaks in June, July, and particularly March of the following year; however, direct sales exhibit a consistent upward trend, achieving their highest in May of the following year, while sales to resellers show fluctuating patterns with the substantial dips around November to January.


## Social Media Trend Analysis

In [47]:
df_social_media = spark.sql("SELECT * FROM AdventureWorks_Lakehouse.Ops_Gold.fact_aisocialreviewanalysis")
df_social_media = df_social_media.withColumn('YearMonth', trim(col('YearMonth')).cast('string'))  
df_social_media = df_social_media.withColumn('YearMonthDate',\
                                                        to_date(concat(col('YearMonth'), lit('-01')), 'yyyy-MM-dd')\
                                                                                                )  
df_social_media.show()

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 54, Finished, Available, Finished)

+---+---------+--------------+----------------+------------+-----------------+--------------------+-------------+
| ID|YearMonth|ProductModelId|ProductModelName|ReviewSource|           LlmKey|            LlmValue|YearMonthDate|
+---+---------+--------------+----------------+------------+-----------------+--------------------+-------------+
|181|  2013-09|            26|        Road-250|     Twitter| AverageSentiment|            Positive|   2013-09-01|
|182|  2013-09|            26|        Road-250|     Twitter|ExtractedInsights|The key points fr...|   2013-09-01|
|183|  2013-09|            26|        Road-250|     Twitter|          Summary|The reviews for t...|   2013-09-01|
|184|  2013-09|            29|      Road-550-W|     Twitter| AverageSentiment|            Positive|   2013-09-01|
|185|  2013-09|            29|      Road-550-W|     Twitter|ExtractedInsights|The reviews highl...|   2013-09-01|
|186|  2013-09|            29|      Road-550-W|     Twitter|          Summary|The review

### Function to analyse product reviews

In [48]:
def SalesProductReviews(socialmedia_query_table):
    system_message = "Analyse the reviews throughout the year of this specific ProductModel. \
                Please analyse the review trend throughout the year. Please highlight the key information.\
                Please look at all three items in the product in each of the month: \
                *AverageSentiment, *ExtractedInsights, *Summary\
                *Do NOT need to Mention the Product ID\
                Please consider if the current view is positive or negative,\
                if the review gets better/worse from the previous Month and previous year,\
                Just provide a summary with the key data including the keys in the review, sentiments and trend.\
                *ONE SENTENCE ONLY*. *NO DETAILED ANALYSIS*."
    sales_review = open_ai_call(system_message = system_message, user_message=socialmedia_query_table)
    return sales_review

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 55, Finished, Available, Finished)

#### Test

In [49]:
if TEST:
    df_social_media_refined = refine_selection(df = df_social_media, col_name="ProductModelName", 
                                            selection=product_model, end_date_str = end_date_str).select('YearMonth', 'LlmKey', 'LlmValue')
    social_media_query_table_string = get_dataframe_show_string(df_social_media_refined,truncate=False)
    sales_review = SalesProductReviews(social_media_query_table_string)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 56, Finished, Available, Finished)

Rate limit error (429) encountered: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 48 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}. Retrying 1/5 after 20 seconds...
Rate limit error (429) encountered: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 22 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}. Retrying 2/5 after 40 seconds...
Throughout the year, the reviews for the multi-colored cycling cap show fluctuations in sentiment, starting positively and highlightin

## Sales Insight

### Function to analyse the sales insight of the product

In [50]:
def SalesInsight(info_for_SalesInsight):
    system_message = "You are reporting sales insight to a sales director.\
    Analyse Insight of the product using the 4 summarised information.\
    Please provide a very very short and concise paragraph summary. Please keep it within 60 words."
    sales_insight = open_ai_call(system_message = system_message, user_message=info_for_SalesInsight)
    return sales_insight

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 57, Finished, Available, Finished)

#### Test

In [51]:
if TEST:
    info_list = [sales_product_vs_category, sales_territory, sales_customer_type, sales_review]
    info_for_SalesInsight = concat_info_with_annotation(info_list)
    sales_insight = SalesInsight(info_for_SalesInsight)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 58, Finished, Available, Finished)

output string from info list:
 information 1:
The Cycling Cap shows inconsistent sales trends with peaks in July 2013 and March 2014, contributing minorly to the overall category sales, which generally have broader variations and reach a peak in June and July 2013 then March 2014.
information 2:
The Cycling Cap sold across various territories from May 2013 to May 2014 with notable sales in the North America and Europe regions; Canada and Southwest US generally had the highest unit sales and revenue, while sales in the Pacific (Australia) also increased notably over time.
information 3:
Throughout the year, sales of the Cycling Cap to resellers generally outperform direct sales, with noticeable peaks in June, July, and particularly March of the following year; however, direct sales exhibit a consistent upward trend, achieving their highest in May of the following year, while sales to resellers show fluctuating patterns with the substantial dips around November to January.
information 4:

## Sales Action

### Function to give sales action

In [75]:
def SalesAction(info_for_SalesAction):
    system_message = "Provide sales action of the product using the following informations\
                The action can be stop/continue production, reduce/promote production,\
                improve manufacturing process, reduce the costs etc\
                *3 BULLET POINTS ONLY*. *NO DETAILED ANALYSIS*. *MAKE IT HTML LIST FORMAT*"
    sales_action = open_ai_call(system_message = system_message, user_message=info_for_SalesAction)
    return sales_action

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 82, Finished, Available, Finished)

#### Test

In [76]:
if TEST:
        info_list = [sales_analysis_previous_month,sales_analysis_previous_year,
                sales_rolling_12_month,sales_year_to_date, sales_summary, 
                sales_product_vs_category, sales_territory, sales_customer_type, 
                sales_review, sales_insight]
        info_for_SalesAction = concat_info_with_annotation(info_list)
        sales_action = SalesAction(info_for_SalesAction)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 83, Finished, Available, Finished)

output string from info list:
 information 1:
In May 2014, the Cycling Cap sold 456 units generating $3294.20 in revenue, compared to 193 units sold and $1735.07 in revenue in April 2014.
information 2:
In May 2014, sales increased to 456 units and revenue reached $3294.20 compared to May 2013 with 261 units and $1295.77 in revenue.
information 3:
Over the past 12 months, sales of Cycling Caps fluctuated reaching a peak in July 2013 and March 2014, with a notable drop in February 2014, while revenue generally followed the same trend as unit sales.
information 4:
Sales for the Sport-100 model saw an upward trend in units sold and revenue from May 2013 to March 2014, with a few dips, peaking in March 2014, but showing a decline in the recent month of May 2014.
information 5:
In May 2014, sales of the Sport-100 model increased significantly to 1241 units, generating $34,956.23 in revenue, showing substantial growth compared to both the previous month and the same month last year. This gro

## Table Saving to Fact_AiSalesAnalysis

In [90]:
product_list = ["Road-250",
                "Road-550-W",
                "Mountain-200",
                "Sport-100",
                "Cycling Cap",
                "Half-Finger Gloves",
                "Long-Sleeve Logo Jersey"]
end_date_list = ['2014-05', '2014-04',
                '2014-03', '2014-02',
                '2014-01',  '2013-12',
                '2013-11', '2013-10',
                '2013-09', '2013-08',
                '2013-07', '2013-06',
                '2013-05']

StatementMeta(, 7ac0edca-d754-4f99-9147-7710749d74e8, 108, Finished, Available, Finished)

In [94]:
from itertools import count
output_data = []
id_counter = count(1)
batch_size = 143

StatementMeta(, 7ac0edca-d754-4f99-9147-7710749d74e8, 112, Finished, Available, Finished)

In [95]:
for product_model in product_list:
    print (product_model)
    for end_date_str in end_date_list:
        print (end_date_str)

        # Calling the Product Model Sale Table with Specific Product and the Period Range of the Sales
        df_filtered = refine_selection(df = df_salesanalysis_final_sorted, col_name = "ProductModel", selection = product_model,end_date_str=end_date_str)
        query_table_string = get_dataframe_show_string(df_filtered, truncate=False) 
        product_model_id = map_model_to_id[product_model]

        # Sales Previous Month
        print ("Sales Previous Month")
        sales_previous_month = SalesPreviousMonth(query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesPreviousMonth",
            LlmValue=sales_previous_month))


        # Sales Previous Year
        print ("Sales Previous Year")
        sales_previous_year = SalesPreviousYear(query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesPreviousYear",
            LlmValue=sales_previous_year))

        # Sales Rolling 12 Months
        print ("Sales Rolling 12 Months")
        sales_rolling_12_months = SalesRolling12Months(query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesRolling12Months",
            LlmValue=sales_rolling_12_months))

        # Sales Year To Date
        print ("Sales Year To Date")
        sales_year_to_date = SalesYearToDate(query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesYearToDate",
            LlmValue=sales_year_to_date))

        # Sales Summary
        print ("Sales Summary")
        info_list = [sales_previous_month,sales_previous_year,sales_rolling_12_months,sales_year_to_date]
        info_for_SalesSummary = concat_info_with_annotation(info_list)
        sales_summary = SalesSummary(info_for_SalesSummary)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesSummary",
            LlmValue=sales_summary))


        # Sales Product Vs Category
        print ("Sales Product Vs Category")
        category_table_string = category_table_AsString(product_model, end_date_str)
        sales_product_vs_category = SalesProductVsCategory(query_table = query_table_string, category_query_table = category_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesProductVsCategory",
            LlmValue=sales_product_vs_category))

        # Sales Territory
        print ("Sales Territory")
        df_territory_refined = refine_selection(df = df_sales_analysis_territory_productmodel, col_name="ProductModel", 
                                            selection=product_model, end_date_str = end_date_str)
        territory_query_table_string = get_dataframe_show_string(df_territory_refined,truncate=False)
        sales_territory = SalesTerritory(territory_query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesTerritory",
            LlmValue=sales_territory))

        # Sales Customer Type
        print ("Sales Customer Type")
        df_sellertype_refined = refine_selection(df = df_sales_product_customers_sellertype, col_name="ProductModel", 
                                            selection=product_model, end_date_str = end_date_str)
        sellertype_query_table_string = get_dataframe_show_string(df_sellertype_refined,truncate=False)
        sales_customer_type = SalesCustomerType(sellertype_query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesCustomerType",
            LlmValue=sales_customer_type))

        # Sales Product Reviews
        print ("Sales Product Reviews")
        df_social_media_refined = refine_selection(df = df_social_media, col_name="ProductModelName", 
                                            selection=product_model, end_date_str = end_date_str).select('YearMonth', 'LlmKey', 'LlmValue')
        social_media_query_table_string = get_dataframe_show_string(df_social_media_refined,truncate=False)
        sales_product_reviews = SalesProductReviews(social_media_query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesProductReviews",
            LlmValue=sales_product_reviews))

        # Sales Insight
        print ("Sales Insight")
        info_list = [sales_product_vs_category, sales_territory, sales_customer_type, sales_product_reviews]
        info_for_SalesInsight = concat_info_with_annotation(info_list)
        sales_insights = SalesInsight(info_for_SalesInsight)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesInsight",
            LlmValue=sales_insights))

        # Sales Action
        print ("Sales Action")
        info_list = [sales_previous_month,sales_previous_year,sales_rolling_12_months,
                sales_year_to_date,sales_summary, 
                sales_product_vs_category, 
                sales_territory, sales_customer_type, 
                sales_product_reviews,sales_insights]
        info_for_SalesAction = concat_info_with_annotation(info_list)
        sales_actions = SalesAction(info_for_SalesAction)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesAction",
            LlmValue=sales_actions))
    
    if len(output_data) >= batch_size:
        # Convert to DataFrame and save the current batch
        batch_df = spark.createDataFrame(output_data)
        batch_df.write.mode("append").saveAsTable("AdventureWorks_Lakehouse.Ops_Gold.Fact_AiSalesAnalysis")
        
        # Clear the batch after saving
        output_data.clear()
        print("Batch saved successfully.")

# Save any remaining entries in output_data if they don’t make up a full batch
if output_data:
    batch_df = spark.createDataFrame(output_data)
    batch_df.write.mode("append").saveAsTable("AdventureWorks_Lakehouse.Ops_Gold.Fact_AiSalesAnalysis")
    print("Final batch saved successfully.")



StatementMeta(, 7ac0edca-d754-4f99-9147-7710749d74e8, 113, Submitted, Running, Running)

Road-250
2014-05
Sales Previous Month
In May 2014, the sales for Road-250 were 216 units with a revenue of $362,593.14, compared to April 2014 where the sales were 43 units with a revenue of $105,064.05.
Sales Previous Year
In May 2014, 216 units were sold generating $362,593.14 in revenue compared to May 2013 where 220 units sold generated $358,043.90 in revenue.
Sales Rolling 12 Months
Over the last 12 months, sales of Road-250 show a cyclical pattern with peaks in June 2013, September 2013, and March 2014, and significantly lower sales in February and April 2014.
Sales Year To Date
Sales of the Road-250 model saw notable fluctuations in the current year, starting with a substantial decrease in February but significantly rebounding in March, with April's sales dropping again but recovering slightly in May.
Sales Summary
output string from info list:
 information 1:
In May 2014, the sales for Road-250 were 216 units with a revenue of $362,593.14, compared to April 2014 where the sales

### Check the output table

In [77]:
df = spark.sql("SELECT * FROM AdventureWorks_Lakehouse.Ops_Gold.fact_aisalesanalysis")
display(df)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 84, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 17b97418-4c6f-4afe-9d6d-6d35d4912948)

In [78]:
# Check the total rows - it should be 1001
df.count()

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 85, Finished, Available, Finished)

924

In [79]:
# Check N.A. cells due to OpenAI connection
df_selected = df[df["YearMonth"] == "2014-05"]
df_selected.show()

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 86, Finished, Available, Finished)

+---+---------+--------------+----------------+------+--------+
| ID|YearMonth|ProductModelId|ProductModelName|LlmKey|LlmValue|
+---+---------+--------------+----------------+------+--------+
+---+---------+--------------+----------------+------+--------+



In [58]:
from pyspark.sql import functions as F
Model_YearMonth_to_Remove = df_selected.groupBy("ProductModelName",).agg(
    F.collect_list("YearMonth").alias("YearMonths"))
Model_YearMonth_to_Remove.show()

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 65, Finished, Available, Finished)

+--------------------+--------------------+
|    ProductModelName|          YearMonths|
+--------------------+--------------------+
|           Sport-100|[2014-05, 2014-05...|
|  Half-Finger Gloves|[2014-05, 2014-05...|
|        Mountain-200|[2014-05, 2014-05...|
|          Road-550-W|[2014-05, 2014-05...|
|         Cycling Cap|[2014-05, 2014-05...|
|            Road-250|[2014-05, 2014-05...|
|Long-Sleeve Logo ...|[2014-05, 2014-05...|
+--------------------+--------------------+



In [63]:
df_filtered = df.join(
    Model_YearMonth_to_Remove.withColumn("YearMonth", F.explode("YearMonths")), 
    on=["ProductModelName", "YearMonth"], 
    how="left_anti"
)
df_filtered.count()

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 70, Finished, Available, Finished)

924

In [61]:
# Double check if any NA rows
df_check = df_filtered[df_filtered["YearMonth"] == "2014-05"]
df_check.show()

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 68, Finished, Available, Finished)

+----------------+---------+---+--------------+------+--------+
|ProductModelName|YearMonth| ID|ProductModelId|LlmKey|LlmValue|
+----------------+---------+---+--------------+------+--------+
+----------------+---------+---+--------------+------+--------+



In [64]:
df_filtered.write.mode("overwrite").saveAsTable("AdventureWorks_Lakehouse.Ops_Gold.Fact_AiSalesAnalysis")

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 71, Finished, Available, Finished)

In [65]:
df_reload = spark.sql("SELECT * FROM AdventureWorks_Lakehouse.Ops_Gold.fact_aisalesanalysis")
df_reload.count()

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 72, Finished, Available, Finished)

924

In [66]:
summary_list = Model_YearMonth_to_Remove.collect()
print (summary_list)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 73, Finished, Available, Finished)

[Row(ProductModelName='Sport-100', YearMonths=['2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05']), Row(ProductModelName='Half-Finger Gloves', YearMonths=['2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05']), Row(ProductModelName='Mountain-200', YearMonths=['2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05']), Row(ProductModelName='Road-550-W', YearMonths=['2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05']), Row(ProductModelName='Cycling Cap', YearMonths=['2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05']), Row(ProductModelName='Road-250', YearMonths=['2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-05', '2014-

In [80]:
from itertools import count
output_data = []
id_counter = count(2000)
batch_size = 11

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 87, Finished, Available, Finished)

In [81]:
for row in summary_list:
    product_model = row["ProductModelName"]
    year_months = list(set(row["YearMonths"]))
    for end_date_str in year_months:
        print (product_model, end_date_str)

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 88, Finished, Available, Finished)

Sport-100 2014-05
Half-Finger Gloves 2014-05
Mountain-200 2014-05
Road-550-W 2014-05
Cycling Cap 2014-05
Road-250 2014-05
Long-Sleeve Logo Jersey 2014-05


In [82]:
for row in summary_list:
    product_model = row["ProductModelName"]
    year_months = list(set(row["YearMonths"]))
    
    for end_date_str in year_months:
        # Calling the Product Model Sale Table with Specific Product and the Period Range of the Sales
        print (product_model, end_date_str)
        df_filtered = refine_selection(df = df_salesanalysis_final_sorted, col_name = "ProductModel", selection = product_model,end_date_str=end_date_str)
        query_table_string = get_dataframe_show_string(df_filtered, truncate=False) 
        product_model_id = map_model_to_id[product_model]

        # Sales Previous Month
        print ("Sales Previous Month")
        sales_previous_month = SalesPreviousMonth(query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesPreviousMonth",
            LlmValue=sales_previous_month))


        # Sales Previous Year
        print ("Sales Previous Year")
        sales_previous_year = SalesPreviousYear(query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesPreviousYear",
            LlmValue=sales_previous_year))

        # Sales Rolling 12 Months
        print ("Sales Rolling 12 Months")
        sales_rolling_12_months = SalesRolling12Months(query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesRolling12Months",
            LlmValue=sales_rolling_12_months))

        # Sales Year To Date
        print ("Sales Year To Date")
        sales_year_to_date = SalesYearToDate(query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesYearToDate",
            LlmValue=sales_year_to_date))

        # Sales Summary
        print ("Sales Summary")
        info_list = [sales_previous_month,sales_previous_year,sales_rolling_12_months,sales_year_to_date]
        info_for_SalesSummary = concat_info_with_annotation(info_list)
        sales_summary = SalesSummary(info_for_SalesSummary)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesSummary",
            LlmValue=sales_summary))


        # Sales Product Vs Category
        print ("Sales Product Vs Category")
        category_table_string = category_table_AsString(product_model, end_date_str)
        sales_product_vs_category = SalesProductVsCategory(query_table = query_table_string, category_query_table = category_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesProductVsCategory",
            LlmValue=sales_product_vs_category))

        # Sales Territory
        print ("Sales Territory")
        df_territory_refined = refine_selection(df = df_sales_analysis_territory_productmodel, col_name="ProductModel", 
                                            selection=product_model, end_date_str = end_date_str)
        territory_query_table_string = get_dataframe_show_string(df_territory_refined,truncate=False)
        sales_territory = SalesTerritory(territory_query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesTerritory",
            LlmValue=sales_territory))

        # Sales Customer Type
        print ("Sales Customer Type")
        df_sellertype_refined = refine_selection(df = df_sales_product_customers_sellertype, col_name="ProductModel", 
                                            selection=product_model, end_date_str = end_date_str)
        sellertype_query_table_string = get_dataframe_show_string(df_sellertype_refined,truncate=False)
        sales_customer_type = SalesCustomerType(sellertype_query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesCustomerType",
            LlmValue=sales_customer_type))

        # Sales Product Reviews
        print ("Sales Product Reviews")
        df_social_media_refined = refine_selection(df = df_social_media, col_name="ProductModelName", 
                                            selection=product_model, end_date_str = end_date_str).select('YearMonth', 'LlmKey', 'LlmValue')
        social_media_query_table_string = get_dataframe_show_string(df_social_media_refined,truncate=False)
        sales_product_reviews = SalesProductReviews(social_media_query_table_string)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesProductReviews",
            LlmValue=sales_product_reviews))

        # Sales Insight
        print ("Sales Insight")
        info_list = [sales_product_vs_category, sales_territory, sales_customer_type, sales_product_reviews]
        info_for_SalesInsight = concat_info_with_annotation(info_list)
        sales_insights = SalesInsight(info_for_SalesInsight)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesInsight",
            LlmValue=sales_insights))

        # Sales Action
        print ("Sales Action")
        info_list = [sales_previous_month,sales_previous_year,sales_rolling_12_months,
                sales_year_to_date,sales_summary, 
                sales_product_vs_category, 
                sales_territory, sales_customer_type, 
                sales_product_reviews,sales_insights]
        info_for_SalesAction = concat_info_with_annotation(info_list)
        sales_actions = SalesAction(info_for_SalesAction)
        output_data.append(Row(
            ID=next(id_counter),
            YearMonth=end_date_str,
            ProductModelId=product_model_id, 
            ProductModelName=product_model,
            LlmKey="SalesAction",
            LlmValue=sales_actions))
    
    if len(output_data) >= batch_size:
        # Convert to DataFrame and save the current batch
        batch_df = spark.createDataFrame(output_data)
        batch_df.write.mode("append").saveAsTable("AdventureWorks_Lakehouse.Ops_Gold.Fact_AiSalesAnalysis")
        
        # Clear the batch after saving
        output_data.clear()
        print("Batch saved successfully.")

# Save any remaining entries in output_data if they don’t make up a full batch
if output_data:
    batch_df = spark.createDataFrame(output_data)
    batch_df.write.mode("append").saveAsTable("AdventureWorks_Lakehouse.Ops_Gold.Fact_AiSalesAnalysis")
    print("Final batch saved successfully.")

StatementMeta(, f4e15ed5-e90e-4464-b89d-aa1ee87f6ba1, 89, Finished, Available, Finished)

Sport-100 2014-05
Sales Previous Month
In May 2014, sales increased to 1241 units sold and $34,956.23 in revenue compared to 654 units sold and $22,855.47 in revenue in April 2014.
Sales Previous Year
In May 2014, the Sport-100 product model sold 1241 units generating $34,956.23 in revenue, compared to 691 units and $9,485.81 revenue in May 2013.
Sales Rolling 12 Months
For the Sport-100 model, total units sold and revenue show an overall increasing trend from May 2013 to May 2014, despite some fluctuations in certain months.
Sales Year To Date
Sales of the Sport-100 model peaked in March 2014 with 1695 units sold and the highest revenue of $43,928.25; there's a fluctuating trend throughout the year with noticeable declines in December 2013 and February 2014.
Sales Summary
output string from info list:
 information 1:
In May 2014, sales increased to 1241 units sold and $34,956.23 in revenue compared to 654 units sold and $22,855.47 in revenue in April 2014.
information 2:
In May 2014, 